In [1]:
import sys
CODEBASE = "./codebase"
if CODEBASE not in sys.path:
    sys.path.append(CODEBASE)

from cooperative_evolution_optimizers import GrayBoxOptimizer
from differential_evolution import DifferentialEvolution as DE

from fitness_functions import FunctionFactory
get_sphere = FunctionFactory.get_sphere

import numpy as np
import matplotlib.pyplot as plt
from time import time

def plot_means_errs(xs, nparray):
    means = np.mean(nparray, axis=-1)
    errs = np.std(nparray, axis=-1)
    plt.errorbar(xs, means, yerr=errs, fmt="--o", capsize=5)

Define $P(x \leq VTR\ |\ N = n)$ as the probability that an optimizer with the sphere function of length $l$ reached fitness $x$ less than or equal to $VTR$ at generation $N = n$ and define $P^k_{GBO}\left(x \leq VTR\ \big|\ N = n\right)$ as the probability that an optimizer with CCGBO with the sphere function of length $l$ for each species reached a cumulative fitness of $x$ less than or equal to $VTR$ by generation $N = n$.

# Study the relationship between GBO and standard optimization

**First hypothesis** on fixed population size:

$$P^k_{GBO}(x \leq VTR\ |\ N = n) \approx P\left(x \leq \frac{VTR}{k}\ \big|\ N = n\right)^k$$

To test this and for varied $k$ we take several $n \leq N_{max}$ and estimate $P(x \leq VTR/k\ |\ N = n)$ over several runs. Then we also estimate $P^k_{GBO}(x \leq VTR\ |\ N = n)$ over several runs and compare the estimated probabilities, not forgetting to take the first one to the $k$-th power.

## Species of length 2, $2 \leq k \leq 6$

In [45]:
Ks = [2,3,4,5,6]
sphere = get_sphere()
population_size = 100
N = 1000
max_generations = 100
VTR = pow(10, -5)

LB = -2
UB = 3
de_args = {"population_size": population_size, "max_generations": max_generations}

estimated_quantiles = np.zeros(max_generations, dtype=np.double)
gbo_quantiles = np.zeros(max_generations, dtype=np.double)
for k in Ks:
    print("k = {}".format(k))
    target_VTR = VTR/k
    ns = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
            
        de = DE(sphere, 2, goal_fitness=target_VTR,
                lower_bounds=LB, upper_bounds=UB, **de_args)
        while not de.has_converged():
            de.evolve()
        ns.append(de.generations())
    ns = np.array(ns)
    # compute some sort of quantiles
    # estimate the right n_k
    for n in range(max_generations):
        estimated_quantiles[n] = sum(ns <= n)/N
    
    print("Estimating the probability for the GBO")
    ns_ = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
        gbo = GrayBoxOptimizer(functions = [sphere]*k,
                              input_spaces = [[2*i, 2*i+1] for i in range(k)],
                              train_partition = [[2*i, 2*i+1] for i in range(k)],
                              lower_bounds = [LB]*2*k, upper_bounds = [UB]*2*k,
                              genetic_algorithms = [DE]*k,
                              genetic_algorithm_arguments = [de_args]*k,
                              goal_fitness = VTR, max_generations=max_generations)
        while not gbo.has_converged():
            gbo.evolve()
        ns_.append(gbo._generations)
    ns_ = np.array(ns_)
    for n in range(max_generations):
        gbo_quantiles[n] = sum(ns_ <= n)/N
    print()
    
    for n in range(1, max_generations):
        eq = estimated_quantiles[n]*100
        p = np.power(estimated_quantiles[n], k)*100
        q = gbo_quantiles[n]*100
        # start only when we have something to show
        if eq == 0 and q == 0:
            continue
        print("n = {:3} || {:7.2f} -> {:7.3f} : {:6.2f} (err {:7.3f})".format(
                n, eq, p, q, p-q))
        # stop if both are already giving 99%
        if q >= 99.9 and p >= 99.9:
            break

k = 2
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   6 ||    0.20 ->   0.000 :   0.00 (err   0.000)
n =   7 ||    0.40 ->   0.002 :   0.00 (err   0.002)
n =   8 ||    0.90 ->   0.008 :   0.00 (err   0.008)
n =   9 ||    2.60 ->   0.068 :   0.00 (err   0.068)
n =  10 ||    5.00 ->   0.250 :   0.20 (err   0.050)
n =  11 ||   10.90 ->   1.188 :   0.80 (err   0.388)
n =  12 ||   18.90 ->   3.572 :   1.80 (err   1.772)
n =  13 ||   30.50 ->   9.303 :   6.20 (err   3.103)
n =  14 ||   46.90 ->  21.996 :  16.50 (err   5.496)
n =  15 ||   64.30 ->  41.345 :  33.00 (err   8.345)
n =  16 ||   80.00 ->  64.000 :  56.20 (err   7.800)
n =  17 ||   89.30 ->  79.745 :  72.50 (err   7.245)
n =  18 ||   94.60 ->  89.492 :  89.00 (err   0.492)
n =  19 ||   98.10 ->  96.236 :  95.70 (err   0.536)
n =  20 ||   99.10 ->  98.208 :  98.70 (err  -0.492)
n =  21 ||   99.70 ->  99.401 :  99.60 (err  -0.199)
n =  22 ||   99.90 ->  99.800 

## Species of length 1, $1 \leq k \leq 16$

In [46]:
Ks = list(range(1, 16+1))
sphere = get_sphere()
population_size = 100
N = 1000
max_generations = 100
VTR = pow(10, -5)

LB = -2
UB = 3
de_args = {"population_size": population_size, "max_generations": max_generations}

estimated_quantiles = np.zeros(max_generations, dtype=np.double)
gbo_quantiles = np.zeros(max_generations, dtype=np.double)
for k in Ks:
    print("k = {}".format(k))
    target_VTR = VTR/k
    ns = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
            
        de = DE(sphere, 1, goal_fitness=target_VTR,
                lower_bounds=LB, upper_bounds=UB, **de_args)
        while not de.has_converged():
            de.evolve()
        ns.append(de.generations())
    ns = np.array(ns)
    # compute some sort of quantiles
    # estimate the right n_k
    for n in range(max_generations):
        estimated_quantiles[n] = sum(ns <= n)/N
    
    print("Estimating the probability for the GBO")
    ns_ = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
        gbo = GrayBoxOptimizer(functions = [sphere]*k,
                              input_spaces = [[i] for i in range(k)],
                              train_partition = [[i] for i in range(k)],
                              lower_bounds = [LB]*1*k, upper_bounds = [UB]*1*k,
                              genetic_algorithms = [DE]*k,
                              genetic_algorithm_arguments = [de_args]*k,
                              goal_fitness = VTR, max_generations=max_generations)
        while not gbo.has_converged():
            gbo.evolve()
        ns_.append(gbo._generations)
    ns_ = np.array(ns_)
    for n in range(max_generations):
        gbo_quantiles[n] = sum(ns_ <= n)/N
    print()
    
    for n in range(1, max_generations):
        eq = estimated_quantiles[n]*100
        p = np.power(estimated_quantiles[n], k)*100
        q = gbo_quantiles[n]*100
        # start only when we have something to show
        if eq == 0 and q == 0:
            continue
        print("n = {:3} || {:7.2f} -> {:7.3f} : {:6.2f} (err {:7.3f})".format(
                n, eq, p, q, p-q))
        # stop if both are already giving 99%
        if q >= 99.9 and p >= 99.9:
            break

k = 1
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||   24.00 ->  24.000 :  11.20 (err  12.800)
n =   2 ||   39.90 ->  39.900 :  20.30 (err  19.600)
n =   3 ||   60.60 ->  60.600 :  35.60 (err  25.000)
n =   4 ||   79.20 ->  79.200 :  58.50 (err  20.700)
n =   5 ||   94.10 ->  94.100 :  79.70 (err  14.400)
n =   6 ||   98.90 ->  98.900 :  93.40 (err   5.500)
n =   7 ||  100.00 -> 100.000 :  99.20 (err   0.800)
n =   8 ||  100.00 -> 100.000 :  99.70 (err   0.300)
n =   9 ||  100.00 -> 100.000 :  99.90 (err   0.100)
k = 2
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||   18.70 ->   3.497 :   1.50 (err   1.997)
n =   2 ||   31.60 ->   9.986 :   4.10 (err   5.886)
n =   3 ||   49.60 ->  24.602 :  11.70 (err  12.902)
n =   4 ||   68.40 ->  46.786 :  30.40 (err  16.386)
n =   5 ||   87.30 ->  76.213 :  61.60 (err  14.613)
n =   6 ||   9

0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    6.30 ->   0.000 :   0.00 (err   0.000)
n =   2 ||   12.80 ->   0.000 :   0.00 (err   0.000)
n =   3 ||   22.50 ->   0.000 :   0.00 (err   0.000)
n =   4 ||   37.20 ->   0.000 :   0.00 (err   0.000)
n =   5 ||   58.60 ->   0.056 :   0.00 (err   0.056)
n =   6 ||   80.00 ->   4.398 :   6.70 (err  -2.302)
n =   7 ||   91.90 ->  30.649 :  58.50 (err -27.851)
n =   8 ||   98.50 ->  80.930 :  94.90 (err -13.970)
n =   9 ||   99.90 ->  98.609 :  99.60 (err  -0.991)
n =  10 ||  100.00 -> 100.000 :  99.90 (err   0.100)
k = 15
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    7.40 ->   0.000 :   0.00 (err   0.000)
n =   2 ||   12.80 ->   0.000 :   0.00 (err   0.000)
n =   3 ||   23.00 ->   0.000 :   0.00 (err   0.000)
n =   4 ||   37.30 ->   0.000 :   0.00 (err   0.000)
n =   5 ||   56.10 

For every small $k$ the approximation doesn't hold, and for "bigger" $k$ we rather get the bound

$$P^k_{GBO}(x \leq VTR\ |\ N = n) \geq P\left(x \leq \frac{VTR}{k}\ \big|\ N = n\right)^k$$

This is because I have been using a rather "easy" $VTR$ and so when some of the species in the GBO hit the value $VTR/k$ before the $n$-th generation, they just keep on improving on behalf of the ones that haven't reached it yet, and so the cumulative fitness gets to $VTR$ earlier than in $n$ generations. If I reduce the $VTR$ a bit I suspect this will no longer happen, so that is what I will do now:

Change $VTR$ to $10^{-6}$. And $10^{-7}$ and $10^{-10}$.

## Species of length 1, $1 \leq k \leq 16$, $VTR = 10^{-6}$.

In [47]:
Ks = list(range(1, 16+1))
sphere = get_sphere()
population_size = 100
N = 1000
max_generations = 100
VTR = pow(10, -6)

LB = -2
UB = 3
de_args = {"population_size": population_size, "max_generations": max_generations}

estimated_quantiles = np.zeros(max_generations, dtype=np.double)
gbo_quantiles = np.zeros(max_generations, dtype=np.double)
for k in Ks:
    print("k = {}".format(k))
    target_VTR = VTR/k
    ns = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
            
        de = DE(sphere, 1, goal_fitness=target_VTR,
                lower_bounds=LB, upper_bounds=UB, **de_args)
        while not de.has_converged():
            de.evolve()
        ns.append(de.generations())
    ns = np.array(ns)
    # compute some sort of quantiles
    # estimate the right n_k
    for n in range(max_generations):
        estimated_quantiles[n] = sum(ns <= n)/N
    
    print("Estimating the probability for the GBO")
    ns_ = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
        gbo = GrayBoxOptimizer(functions = [sphere]*k,
                              input_spaces = [[i] for i in range(k)],
                              train_partition = [[i] for i in range(k)],
                              lower_bounds = [LB]*1*k, upper_bounds = [UB]*1*k,
                              genetic_algorithms = [DE]*k,
                              genetic_algorithm_arguments = [de_args]*k,
                              goal_fitness = VTR, max_generations=max_generations)
        while not gbo.has_converged():
            gbo.evolve()
        ns_.append(gbo._generations)
    ns_ = np.array(ns_)
    for n in range(max_generations):
        gbo_quantiles[n] = sum(ns_ <= n)/N
    print()
    
    for n in range(1, max_generations):
        eq = estimated_quantiles[n]*100
        p = np.power(estimated_quantiles[n], k)*100
        q = gbo_quantiles[n]*100
        # start only when we have something to show
        if eq == 0 and q == 0:
            continue
        print("n = {:3} || {:7.2f} -> {:7.3f} : {:6.2f} (err {:7.3f})".format(
                n, eq, p, q, p-q))
        # stop if both are already giving 99%
        if q >= 99.9 and p >= 99.9:
            break

k = 1
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    7.50 ->   7.500 :   3.60 (err   3.900)
n =   2 ||   13.70 ->  13.700 :   7.00 (err   6.700)
n =   3 ||   25.00 ->  25.000 :  12.50 (err  12.500)
n =   4 ||   41.50 ->  41.500 :  25.10 (err  16.400)
n =   5 ||   62.70 ->  62.700 :  43.40 (err  19.300)
n =   6 ||   82.70 ->  82.700 :  64.90 (err  17.800)
n =   7 ||   94.60 ->  94.600 :  83.10 (err  11.500)
n =   8 ||   98.40 ->  98.400 :  95.40 (err   3.000)
n =   9 ||   99.50 ->  99.500 :  98.60 (err   0.900)
n =  10 ||   99.80 ->  99.800 :  99.80 (err   0.000)
n =  11 ||  100.00 -> 100.000 :  99.90 (err   0.100)
k = 2
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    5.30 ->   0.281 :   0.40 (err  -0.119)
n =   2 ||    9.80 ->   0.960 :   0.90 (err   0.060)
n =   3 ||   18.00 ->   3.240 :   2.00 (err   1.240)
n =   4 ||   3

0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    2.20 ->   0.000 :   0.00 (err   0.000)
n =   2 ||    5.60 ->   0.000 :   0.00 (err   0.000)
n =   3 ||    8.00 ->   0.000 :   0.00 (err   0.000)
n =   4 ||   14.20 ->   0.000 :   0.00 (err   0.000)
n =   5 ||   27.20 ->   0.000 :   0.00 (err   0.000)
n =   6 ||   44.00 ->   0.005 :   0.00 (err   0.005)
n =   7 ||   67.00 ->   0.818 :   0.60 (err   0.218)
n =   8 ||   84.50 ->  13.252 :  17.90 (err  -4.648)
n =   9 ||   95.00 ->  54.036 :  71.80 (err -17.764)
n =  10 ||   98.70 ->  85.468 :  96.40 (err -10.932)
n =  11 ||   99.70 ->  96.459 :  99.60 (err  -3.141)
n =  12 ||  100.00 -> 100.000 :  99.90 (err   0.100)
k = 13
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    2.60 ->   0.000 :   0.00 (err   0.000)
n =   2 ||    4.50 ->   0.000 :   0.00 (err   0.000)
n =   3 ||    8.70 

## Species of length 1, $1 \leq k \leq 16$, $VTR = 10^{-7}$.

In [48]:
Ks = list(range(1, 16+1))
sphere = get_sphere()
population_size = 100
N = 1000
max_generations = 100
VTR = pow(10, -7)

LB = -2
UB = 3
de_args = {"population_size": population_size, "max_generations": max_generations}

estimated_quantiles = np.zeros(max_generations, dtype=np.double)
gbo_quantiles = np.zeros(max_generations, dtype=np.double)
for k in Ks:
    print("k = {}".format(k))
    target_VTR = VTR/k
    ns = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
            
        de = DE(sphere, 1, goal_fitness=target_VTR,
                lower_bounds=LB, upper_bounds=UB, **de_args)
        while not de.has_converged():
            de.evolve()
        ns.append(de.generations())
    ns = np.array(ns)
    # compute some sort of quantiles
    # estimate the right n_k
    for n in range(max_generations):
        estimated_quantiles[n] = sum(ns <= n)/N
    
    print("Estimating the probability for the GBO")
    ns_ = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
        gbo = GrayBoxOptimizer(functions = [sphere]*k,
                              input_spaces = [[i] for i in range(k)],
                              train_partition = [[i] for i in range(k)],
                              lower_bounds = [LB]*1*k, upper_bounds = [UB]*1*k,
                              genetic_algorithms = [DE]*k,
                              genetic_algorithm_arguments = [de_args]*k,
                              goal_fitness = VTR, max_generations=max_generations)
        while not gbo.has_converged():
            gbo.evolve()
        ns_.append(gbo._generations)
    ns_ = np.array(ns_)
    for n in range(max_generations):
        gbo_quantiles[n] = sum(ns_ <= n)/N
    print()
    
    for n in range(1, max_generations):
        eq = estimated_quantiles[n]*100
        p = np.power(estimated_quantiles[n], k)*100
        q = gbo_quantiles[n]*100
        # start only when we have something to show
        if eq == 0 and q == 0:
            continue
        print("n = {:3} || {:7.2f} -> {:7.3f} : {:6.2f} (err {:7.3f})".format(
                n, eq, p, q, p-q))
        # stop if both are already giving 99%
        if q >= 99.9 and p >= 99.9:
            break

k = 1
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    2.70 ->   2.700 :   1.40 (err   1.300)
n =   2 ||    5.50 ->   5.500 :   2.20 (err   3.300)
n =   3 ||    9.80 ->   9.800 :   5.40 (err   4.400)
n =   4 ||   16.50 ->  16.500 :  10.10 (err   6.400)
n =   5 ||   28.80 ->  28.800 :  16.80 (err  12.000)
n =   6 ||   48.60 ->  48.600 :  29.40 (err  19.200)
n =   7 ||   69.50 ->  69.500 :  49.20 (err  20.300)
n =   8 ||   87.20 ->  87.200 :  68.80 (err  18.400)
n =   9 ||   96.30 ->  96.300 :  86.70 (err   9.600)
n =  10 ||   99.40 ->  99.400 :  94.70 (err   4.700)
n =  11 ||  100.00 -> 100.000 :  98.80 (err   1.200)
n =  12 ||  100.00 -> 100.000 :  99.80 (err   0.200)
n =  13 ||  100.00 -> 100.000 :  99.90 (err   0.100)
k = 2
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    1.20 ->   0.014 :   0.00 (err   0.014)
n =   2 ||    

0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    0.50 ->   0.000 :   0.00 (err   0.000)
n =   2 ||    1.60 ->   0.000 :   0.00 (err   0.000)
n =   3 ||    2.80 ->   0.000 :   0.00 (err   0.000)
n =   4 ||    5.40 ->   0.000 :   0.00 (err   0.000)
n =   5 ||   10.40 ->   0.000 :   0.00 (err   0.000)
n =   6 ||   19.00 ->   0.000 :   0.00 (err   0.000)
n =   7 ||   33.10 ->   0.001 :   0.00 (err   0.001)
n =   8 ||   50.50 ->   0.054 :   0.00 (err   0.054)
n =   9 ||   70.00 ->   1.977 :   1.50 (err   0.477)
n =  10 ||   85.70 ->  18.314 :  30.60 (err -12.286)
n =  11 ||   97.10 ->  72.346 :  79.10 (err  -6.754)
n =  12 ||   99.30 ->  92.564 :  97.50 (err  -4.936)
n =  13 ||   99.80 ->  97.822 :  99.40 (err  -1.578)
n =  14 ||  100.00 -> 100.000 :  99.90 (err   0.100)
k = 12
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   1 ||    0.60 

## Species of length 1, $6 \leq k \leq 10$, $VTR = 10^{-10}$.

In [51]:
Ks = list(range(6, 10+1))
sphere = get_sphere()
population_size = 100
N = 1000
max_generations = 100
VTR = pow(10, -10)

LB = -2
UB = 3
de_args = {"population_size": population_size, "max_generations": max_generations}

estimated_quantiles = np.zeros(max_generations, dtype=np.double)
gbo_quantiles = np.zeros(max_generations, dtype=np.double)
for k in Ks:
    print("k = {}".format(k))
    target_VTR = VTR/k
    ns = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
            
        de = DE(sphere, 1, goal_fitness=target_VTR,
                lower_bounds=LB, upper_bounds=UB, **de_args)
        while not de.has_converged():
            de.evolve()
        ns.append(de.generations())
    ns = np.array(ns)
    # compute some sort of quantiles
    # estimate the right n_k
    for n in range(max_generations):
        estimated_quantiles[n] = sum(ns <= n)/N
    
    print("Estimating the probability for the GBO")
    ns_ = []
    for i in range(N):
        if (i % 100) == 0:
            print(i, end = " ")
        gbo = GrayBoxOptimizer(functions = [sphere]*k,
                              input_spaces = [[i] for i in range(k)],
                              train_partition = [[i] for i in range(k)],
                              lower_bounds = [LB]*1*k, upper_bounds = [UB]*1*k,
                              genetic_algorithms = [DE]*k,
                              genetic_algorithm_arguments = [de_args]*k,
                              goal_fitness = VTR, max_generations=max_generations)
        while not gbo.has_converged():
            gbo.evolve()
        ns_.append(gbo._generations)
    ns_ = np.array(ns_)
    for n in range(max_generations):
        gbo_quantiles[n] = sum(ns_ <= n)/N
    print()
    
    for n in range(1, max_generations):
        eq = estimated_quantiles[n]*100
        p = np.power(estimated_quantiles[n], k)*100
        q = gbo_quantiles[n]*100
        # start only when we have something to show
        if eq == 0 and q == 0:
            continue
        print("n = {:3} || {:7.2f} -> {:7.3f} : {:6.2f} (err {:7.3f})".format(
                n, eq, p, q, p-q))
        # stop if both are already giving 99%
        if q >= 99.9 and p >= 99.9:
            break

k = 6
0 100 200 300 400 500 600 700 800 900 Estimating the probability for the GBO
0 100 200 300 400 500 600 700 800 900 
n =   2 ||    0.10 ->   0.000 :   0.00 (err   0.000)
n =   3 ||    0.10 ->   0.000 :   0.00 (err   0.000)
n =   4 ||    0.20 ->   0.000 :   0.00 (err   0.000)
n =   5 ||    0.70 ->   0.000 :   0.00 (err   0.000)
n =   6 ||    1.50 ->   0.000 :   0.00 (err   0.000)
n =   7 ||    2.40 ->   0.000 :   0.00 (err   0.000)
n =   8 ||    3.40 ->   0.000 :   0.00 (err   0.000)
n =   9 ||    5.50 ->   0.000 :   0.00 (err   0.000)
n =  10 ||   11.20 ->   0.000 :   0.00 (err   0.000)
n =  11 ||   20.10 ->   0.007 :   0.00 (err   0.007)
n =  12 ||   33.40 ->   0.139 :   0.20 (err  -0.061)
n =  13 ||   51.30 ->   1.823 :   0.80 (err   1.023)
n =  14 ||   69.90 ->  11.664 :  10.30 (err   1.364)
n =  15 ||   84.00 ->  35.130 :  41.90 (err  -6.770)
n =  16 ||   93.50 ->  66.814 :  78.60 (err -11.786)
n =  17 ||   98.00 ->  88.584 :  95.30 (err  -6.716)
n =  18 ||   99.80 ->  98.806 

## The approximation does not hold

Check the assumptions.

So far, I've only got this

$$P^k_{GBO}(x \leq VTR\ |\ N = n) \geq P\left(x \leq \frac{VTR}{k}\ \big|\ N = n\right)^k$$

Do we maybe have

$$P^k_{GBO}(x \leq VTR\ |\ N = n) \approx P\left(x \leq VTR\ \big|\ N = n\right)^k$$

?